In [4]:
#Please use python 3.5 or above
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Input, Concatenate
from keras import optimizers
from keras.models import load_model, Model
import pandas as pd
import emoji
import json, argparse, os
import re
import io
import sys
import emoji
import re

In [107]:
strng = 'Money money and lots of money 😍  😍  <eos> I need to get it tailored but Im in love with it  😍  <eos>  😁 😁'
remove_text(strng)

['😍', '😍', '😍', '😁', '😁']

In [98]:
def is_emoji(s):
    return s in emoji.UNICODE_EMOJI

def remove_emoji(text):
    letter_array = [char for char in text if not is_emoji(char)]    
    return ''.join(letter_array)

def add_space(text):
    return ''.join(' ' + char + ' ' if is_emoji(char) else char for char in text).strip()
a
def remove_text(text):
    words = text.split(' ')
    emojis = [word for word in words if is_emoji(word)]
    return emojis

def count_length(text):
    return len(text)

def count_upper_case(text):
    return sum(1 for c in text if c.isupper())

def find_converage(big_dict, small_dict):
    not_in_small = {}
    missing = 0
    exist_both = 0
    for key in small_dict:
        if key in big_dict:
            exist_both += 1
        else:
            not_in_small[key] = 0
            missing += 1
    coverage = exist_both/(exist_both + missing)
    return [coverage, missing, exist_both], not_in_small

def add_word(string, word):
    return 1 if word in string.split(' ') else 0

def find_number_missing(not_in_small, tokenizer_input_data):
    loop_count = 0
    for line in tokenizer_input_data:
        for key, _ in not_in_small.items():
            not_in_small[key] += add_word(line, key)
        loop_count += 1        
    return not_in_small

def fix_apos(word):
    replaced_word = re.sub(r"(it['|´|’]s)", "it is", word)
    replaced_word = re.sub(r"(can['|´|’]t)", "can not", replaced_word)
    replaced_word = re.sub(r"(ain['|´|’]t)", "am not", replaced_word)
    replaced_word = re.sub(r"([I|a-z]{0,10})['|´|’]re", r"\1 are", replaced_word)
    replaced_word = re.sub(r"([I|a-z]{0,10})['|´|’]ll", r"\1 will", replaced_word)
    replaced_word = re.sub(r"([I|a-z]{0,10})['|´|’]ve", r"\1 have", replaced_word)
    replaced_word = re.sub(r"(he|who|how|when|there)['|´|’]s", r"\1 is", replaced_word)
    replaced_word = re.sub(r"that['|´|’]s", r"that is", replaced_word)
    replaced_word = re.sub(r"what['|´|’]s", r"what is", replaced_word)
    replaced_word = re.sub(r"(let['|´|’]s)", "let us", replaced_word)
    replaced_word = re.sub(r"([I|i]['|´|’]m)", "i am", replaced_word)
    replaced_word = re.sub(r"(won['|´|’]t)", "will not", replaced_word)
    replaced_word = re.sub(r"(n['|´|’]t)", " not", replaced_word)
    replaced_word = re.sub(r"['|´|’]", r" ", replaced_word)
    return replaced_word

def str2emoji(text):
    text = re.sub(r":‑\)"," ☺️ ",text)
    text = re.sub(r":\)\)\)\)"," ☺️ ",text)
    text = re.sub(r":\)\)\)"," ☺️ ",text)
    text = re.sub(r":\)\)"," ☺️ ",text)
    text = re.sub(r":\)"," ☺️ ",text)
    text = re.sub(r":-\]"," ☺️ ",text)
    text = re.sub(r":\]"," ☺️ ",text)
    text = re.sub(r":-3"," ☺️ ",text)
    text = re.sub(r":3","  ☺️ ",text)
    text = re.sub(r":->"," ☺️ ",text)
    text = re.sub(r"</3",' 💔 ',text)
    text = re.sub(r"<3"," ❤️ ",text)
    text = re.sub(r":>"," ☺️ ",text)
    text = re.sub(r"8-\)"," ☺️ ",text)
    text = re.sub(r":o\)"," ☺️ ",text)
    text = re.sub(r":-\}"," ☺️ ",text)
    text = re.sub(r":\}"," ☺️ ",text)
    text = re.sub(r":-\)"," ☺️ ",text)
    text = re.sub(r":c\)"," ☺️ ",text)
    text = re.sub(r":\^\)"," ☺️ ",text)
    text = re.sub(r"=\]"," ☺️ ",text)
    text = re.sub(r"=\)"," ☺️ ",text)
    text = re.sub(r":‑D"," 😃 ",text)
    text = re.sub(r":D"," 😃 ",text)
    text = re.sub(r"8‑D"," 😃 ",text)
    text = re.sub(r"8D"," 😃 ",text)
    text = re.sub(r"X‑D"," 😃 ",text)
    text = re.sub(r"XD"," 😃 ",text)
    text = re.sub(r"=D"," 😃 ",text)
    text = re.sub(r"=3"," 😃 ",text)
    text = re.sub(r"B\^D"," 😃 ",text)
    text = re.sub(r":-\)\)"," 😃 ",text)
    text = re.sub(r":‑\("," ☹️ ",text)
    text = re.sub(r":-\("," ☹️ ",text)
    text = re.sub(r":-d", " 🤤 ",text)
    text = re.sub(r":d", " 🤤 ",text)
    text = re.sub(r"=d", " 🤤 ",text)

    text = re.sub(r":\("," ☹️ ",text)
    text = re.sub(r":‑c"," ☹️ ",text)
    text = re.sub(r":c"," ☹️ ",text)
    text = re.sub(r":‑<"," ☹️ ",text)
    text = re.sub(r":<"," ☹️ ",text)
    text = re.sub(r":‑\["," ☹️ ",text)
    text = re.sub(r":\["," ☹️ ",text)
    text = re.sub(r":-\|\|"," ☹️ ",text)
    text = re.sub(r">:\["," ☹️ ",text)
    text = re.sub(r":\{"," ☹️ ",text)
    text = re.sub(r":@"," ☹️ ",text)
    text = re.sub(r">:\("," ☹️ ",text)
    text = re.sub(r":'‑\("," 😭 ",text)
    text = re.sub(r":'\("," 😭 ",text)
    text = re.sub(r":'‑\)"," 😃 ",text)
    text = re.sub(r":'\)"," 😃 ",text)
    text = re.sub(r"D‑':"," 😧 ",text)
    text = re.sub(r"D:<"," 😨 ",text)
    text = re.sub(r"D:"," 😧 ",text)
    text = re.sub(r"D8"," 😧 ",text)
    text = re.sub(r"D;"," 😧 ",text)
    text = re.sub(r"D="," 😧 ",text)
    text = re.sub(r"DX"," 😧 ",text)
    text = re.sub(r":‑O"," 😮 ",text)
    text = re.sub(r":O"," 😮 ",text)
    text = re.sub(r":‑o"," 😮 ",text)
    text = re.sub(r":o"," 😮 ",text)
    text = re.sub(r":-0"," 😮 ",text)
    text = re.sub(r"8‑0"," 😮 ",text)
    text = re.sub(r">:O"," 😮 ",text)
    text = re.sub(r":-\*"," 😗 ",text)
    text = re.sub(r":\*"," 😗 ",text)
    text = re.sub(r":X"," 😗 ",text)
    text = re.sub(r";‑\)"," 😉 ",text)
    text = re.sub(r";\)"," 😉 ",text)
    text = re.sub(r"\*-\)"," 😉 ",text)
    text = re.sub(r"\*\)"," 😉 ",text)
    text = re.sub(r";‑\]"," 😉 ",text)
    text = re.sub(r";\]"," 😉 ",text)
    text = re.sub(r";\^\)"," 😉 ",text)
    text = re.sub(r":‑,"," 😉 ",text)
    text = re.sub(r";D"," 😉 ",text)
    text = re.sub(r":‑P"," 😛 ",text)
    text = re.sub(r":‑p"," 😛 ",text)
    text = re.sub(r":P"," 😛 ",text)
    text = re.sub(r":p"," 😛 ",text)
    text = re.sub(r"X‑P"," 😛 ",text)
    text = re.sub(r":‑Þ"," 😛 ",text)
    text = re.sub(r":Þ"," 😛 ",text)
    text = re.sub(r":b"," 😛 ",text)
    text = re.sub(r"d:"," 😛 ",text)
    text = re.sub(r"=p"," 😛 ",text)
    text = re.sub(r">:P"," 😛 ",text)
    text = re.sub(r":‑/"," 😕 ",text)
    text = re.sub(r":/"," 😕 ",text)
    text = re.sub(r":-\[\.\]"," 😕 ",text)
    text = re.sub(r">:/"," 😕 ",text)
    text = re.sub(r"=/"," 😕 ",text)
    text = re.sub(r":L"," 😕 ",text)
    text = re.sub(r"=L"," 😕 ",text)
    text = re.sub(r":S"," 😕 ",text)
    text = re.sub(r":‑\|"," 😐 ",text)
    text = re.sub(r":\|"," 😐 ",text)
    text = re.sub(r":$"," 😳 ",text)
    text = re.sub(r":‑x"," 🤐 ",text)
    text = re.sub(r":x"," 🤐 ",text)
    text = re.sub(r":‑#"," 🤐 ",text)
    text = re.sub(r":#"," 🤐 ",text)
    text = re.sub(r":‑&"," 🤐 ",text)
    text = re.sub(r":&"," 🤐 ",text)
    text = re.sub(r"O:‑\)"," 😇 ",text)
    text = re.sub(r"O:\)"," 😇 ",text)
    text = re.sub(r"0:‑3"," 😇 ",text)
    text = re.sub(r"0:3"," 😇 ",text)
    text = re.sub(r"0:‑\)"," 😇 ",text)
    text = re.sub(r"0:\)"," 😇 ",text)
    text = re.sub(r":‑b"," 😛 ",text)
    text = re.sub(r"0;\^\)"," 😇 ",text)
    text = re.sub(r">:‑\)"," 😈 ",text)
    text = re.sub(r">:\)"," 😈 ",text)
    text = re.sub(r"\}:‑\)"," 😈 ",text)
    text = re.sub(r"\}:\)"," 😈 ",text)
    text = re.sub(r"3:‑\)"," 😈 ",text)
    text = re.sub(r"3:\)"," 😈 ",text)
    text = re.sub(r">;\)"," 😈 ",text)
    text = re.sub(r"\|;‑\)"," 😎 ",text)
    text = re.sub(r"\|‑O"," 😏 ",text)
    text = re.sub(r":‑J"," 😏 ",text)
    text = re.sub(r"%‑\)"," 😵 ",text)
    text = re.sub(r"%\)"," 😵 ",text)
    text = re.sub(r":-###.."," 🤒 ",text)
    text = re.sub(r":###.."," 🤒 ",text)
    text = re.sub(r"\(>_<\)"," 😣 ",text)
    text = re.sub(r"\(>_<\)>"," 😣 ",text)
    text = re.sub(r"\(';'\)"," 👶 ",text)
    text = re.sub(r"\(\^\^>``"," 😓 ",text)
    text = re.sub(r"\(\^_\^;\)"," 😓 ",text)
    text = re.sub(r"\(-_-;\)"," 😓 ",text)
    text = re.sub(r"\(~_~;\) \(・\.・;\)"," 😓 ",text)
    text = re.sub(r"\(-_-\)zzz"," 😴 ",text)
    text = re.sub(r"\(\^_-\)"," 😉 ",text)
    text = re.sub(r"\(\(\+_\+\)\)"," 😕 ",text)
    text = re.sub(r"\(\+o\+\)"," 😕 ",text)
    text = re.sub(r"\^_\^"," 😃 ",text)
    text = re.sub(r"\(\^_\^\)/"," 😃 ",text)
    text = re.sub(r"\(\^O\^\)／"," 😃 ",text)
    text = re.sub(r"\(\^o\^\)／"," 😃 ",text)
    text = re.sub(r"\(__\)"," 🙇 ",text)
    text = re.sub(r"_\(\._\.\)_"," 🙇 ",text)
    text = re.sub(r"<\(_ _\)>"," 🙇 ",text)
    text = re.sub(r"<m\(__\)m>"," 🙇 ",text)
    text = re.sub(r"m\(__\)m"," 🙇 ",text)
    text = re.sub(r"m\(_ _\)m"," 🙇 ",text)
    text = re.sub(r"\('_'\)"," 😭 ",text)
    text = re.sub(r"\(/_;\)"," 😭 ",text)
    text = re.sub(r"\(T_T\) \(;_;\)"," 😭 ",text)
    text = re.sub(r"\(;_;"," 😭 ",text)
    text = re.sub(r"\(;_:\)"," 😭 ",text)
    text = re.sub(r"\(;O;\)"," 😭 ",text)
    text = re.sub(r"\(:_;\)"," 😭 ",text)
    text = re.sub(r"\(ToT\)"," 😭 ",text)
    text = re.sub(r";_;"," 😭 ",text)
    text = re.sub(r";-;"," 😭 ",text)
    text = re.sub(r";n;"," 😭 ",text)
    text = re.sub(r";;"," 😭 ",text)
    text = re.sub(r"Q\.Q"," 😭 ",text)
    text = re.sub(r"T\.T"," 😭 ",text)
    text = re.sub(r"QQ"," 😭 ",text)
    text = re.sub(r"Q_Q"," 😭 ",text)
    text = re.sub(r"\(-\.-\)"," 😞 ",text)
    text = re.sub(r"\(-_-\)"," 😞 ",text)
    text = re.sub(r"-_-"," 😞 ",text)
    text = re.sub(r"\(一一\)"," 😞 ",text)
    text = re.sub(r"\(；一_一\)"," 😞 ",text)
    text = re.sub(r"\(=_=\)"," 😩 ",text)
    text = re.sub(r"\(=\^\·\^=\)"," 😺 ",text)
    text = re.sub(r"\(=\^\·\·\^=\)"," 😺 ",text)
    text = re.sub(r"=_\^= "," 😺 ",text)
    text = re.sub(r"\(\.\.\)"," 😔 ",text)
    text = re.sub(r"\(\._\.\)"," 😔 ",text)
    text = re.sub(r"\(\・\・\?"," 😕 ",text)
    text = re.sub(r"\(\?_\?\)"," 😕 ",text)
    text = re.sub(r">\^_\^<"," 😃 ",text)
    text = re.sub(r"<\^!\^>"," 😃 ",text)
    text = re.sub(r"\^/\^"," 😃 ",text)
    text = re.sub(r"\（\*\^_\^\*）"," 😃 ",text)
    text = re.sub(r"\(\^<\^\) \(\^\.\^\)"," 😃 ",text)
    text = re.sub(r"\(^\^\)"," 😃 ",text)
    text = re.sub(r"\(\^\.\^\)"," 😃 ",text)
    text = re.sub(r"\(\^_\^\.\)"," 😃 ",text)
    text = re.sub(r"\(\^_\^\)"," 😃 ",text)
    text = re.sub(r"\(\^\^\)"," 😃 ",text)
    text = re.sub(r"\(\^J\^\)"," 😃 ",text)
    text = re.sub(r"\(\*\^\.\^\*\)"," 😃 ",text)
    text = re.sub(r"\(\^—\^\）"," 😃 ",text)
    text = re.sub(r"\(#\^\.\^#\)"," 😃 ",text)
    text = re.sub(r"\（\^—\^\）"," 👋 ",text)
    text = re.sub(r"\(;_;\)/~~~"," 👋 ",text)
    text = re.sub(r"\(\^\.\^\)/~~~"," 👋 ",text)
    text = re.sub(r"\(T_T\)/~~~"," 👋 ",text)
    text = re.sub(r"\(\*\^0\^\*\)"," 😍 ",text)
    text = re.sub(r"\(\*_\*\)"," 😍 ",text)
    text = re.sub(r"\(\*_\*;"," 😍 ",text)
    text = re.sub(r"\(\+_\+\) \(@_@\)"," 😍 ",text)
    text = re.sub(r"\(\*\^\^\)v"," 😂 ",text)
    text = re.sub(r"\(\^_\^\)v"," 😂 ",text)
    text = re.sub(r"\(ーー;\)"," 😓 ",text)
    text = re.sub(r"\(\^0_0\^\)"," 😎 ",text)
    text = re.sub(r"\(\＾ｖ\＾\)"," 😀 ",text)
    text = re.sub(r"\(\＾ｕ\＾\)"," 😀 ",text)
    text = re.sub(r"\(\^\)o\(\^\)"," 😀 ",text)
    text = re.sub(r"\(\^O\^\)"," 😀 ",text)
    text = re.sub(r"\(\^o\^\)"," 😀 ",text)
    text = re.sub(r"\)\^o\^\("," 😀 ",text)
    text = re.sub(r":O o_O"," 😮 ",text)
    text = re.sub(r"o_0"," 😮 ",text)
    text = re.sub(r"o\.O"," 😮 ",text)
    text = re.sub(r"\(o\.o\)"," 😮 ",text)
    text = re.sub(r"oO"," 😮 ",text)
    text = re.sub(r':\‑\)','😃',text)
    text = re.sub(r":\)"," ☺️ ",text)
    text = re.sub(r":-]"," ☺️ ",text)
    text = re.sub(r":]"," ☺️ ",text)
    text = re.sub(r"8\-\)"," ☺️ ",text)
    text = re.sub(r":o\)"," ☺️ ",text)
    text = re.sub(r":-}"," ☺️ ",text)
    text = re.sub(r":}"," ☺️ ",text)
    text = re.sub(r":\-\)"," ☺️ ",text)
    text = re.sub(r":c\)"," ☺️ ",text)
    text = re.sub(r":^\)"," ☺️ ",text)
    text = re.sub(r"=]"," ☺️ ",text)
    text = re.sub(r"=\)"," ☺️ ",text)
    text = re.sub(r"B^D"," 😃 ",text)
    text = re.sub(r":-\)\)"," 😃 ",text)
    text = re.sub(r":-\("," ☹️ ",text)

    text = re.sub(r":‑\("," ☹️ ",text)
    text = re.sub(r":\("," ☹️ ",text)
    text = re.sub(r":\‑\["," ☹️ ",text)
    text = re.sub(r":\["," ☹️ ",text)
    text = re.sub(r":-\|\|"," ☹️ ",text)
    text = re.sub(r">\:\["," ☹️ ",text)
    text = re.sub(r":{"," ☹️ ",text)
    text = re.sub(r">\:\("," ☹️ ",text)
    text = re.sub(r":'‑\("," 😭 ",text)
    text = re.sub(r":'\("," 😭 ",text)
    text = re.sub(r":'\‑\)"," 😃 ",text)
    text = re.sub(r":'\)"," 😃 ",text)
    text = re.sub(r":-\*"," 😗 ",text)
    text = re.sub(r":\*"," 😗 ",text)
    text = re.sub(r";\‑\)"," 😉 ",text)
    text = re.sub(r";\)"," 😉 ",text)
    text = re.sub(r"\*\-\)"," 😉 ",text)
    text = re.sub(r"\*\)"," 😉 ",text)
    text = re.sub(r";‑\]"," 😉 ",text)
    text = re.sub(r";\]"," 😉 ",text)
    text = re.sub(r";^\)"," 😉 ",text)
    text = re.sub(r">\:\[\(\)\]"," 😕 ",text)

    text = re.sub(r":\[\(\)\]"," 😕 ",text)
    text = re.sub(r"=\[\(\)\]"," 😕 ",text)
    text = re.sub(r":‑\|"," 😐 ",text)
    text = re.sub(r":\|"," 😐 ",text)
    text = re.sub(r"O:‑\)"," 😇 ",text)
    text = re.sub(r"O:\)"," 😇 ",text)
    text = re.sub(r"0:‑\)"," 😇 ",text)
    text = re.sub(r"0:\)"," 😇 ",text)
    text = re.sub(r"0;^\)"," 😇 ",text)
    text = re.sub(r">:‑\)"," 😈 ",text)
    text = re.sub(r">:\)"," 😈 ",text)
    text = re.sub(r"}:‑\)"," 😈 ",text)
    text = re.sub(r"}:\)"," 😈 ",text)
    text = re.sub(r"3:‑\)"," 😈 ",text)
    text = re.sub(r"3:\)"," 😈 ",text)
    text = re.sub(r">;\)"," 😈 ",text)
    text = re.sub(r"\|;‑\)"," 😎 ",text)
    text = re.sub(r"\|‑O"," 😏 ",text)
    text = re.sub(r"%‑\)"," 😵 ",text)
    text = re.sub(r"%\)"," 😵 ",text)
    text = re.sub(r"\(>_<\)"," 😣 ",text)
    text = re.sub(r"\(>_<\)>"," 😣 ",text)
    text = re.sub(r"\(';'\)"," Baby ",text)
    text = re.sub(r"\(^^>``"," 😓 ",text)
    text = re.sub(r"\(^_^;\)"," 😓 ",text)
    text = re.sub(r"\(-_-;\)"," 😓 ",text)

    text = re.sub(r"\(~_~;\) \(・\.・;\)"," 😓 ",text)
    text = re.sub(r"\(-_-\)zzz"," 😴 ",text)
    text = re.sub(r"\(^_-\)"," 😉 ",text)
    text = re.sub(r"\(\(\+_\+\)\)"," 😕 ",text)
    text = re.sub(r"\(\+o\+\)"," 😕 ",text)
    text = re.sub(r"^_^"," 😃 ",text)
    text = re.sub(r"\(^_^\)/"," 😃 ",text)
    text = re.sub(r"\(^O^\)／"," 😃 ",text)
    text = re.sub(r"\(__\)"," 🙇 ",text)
    text = re.sub(r"_\(._.\)_"," 🙇 ",text)
    text = re.sub(r"<\(_ _\)>"," 🙇 ",text)
    text = re.sub(r"<m\(__\)m>"," 🙇 ",text)
    text = re.sub(r"m\(__\)m"," 🙇 ",text)
    text = re.sub(r"m\(_ _\)m"," 🙇 ",text)
    text = re.sub(r"\('_'\)"," 😭 ",text)
    text = re.sub(r"\(/_;\)"," 😭 ",text)
    text = re.sub(r"\(T_T\) (;_;)"," 😭 ",text)
    text = re.sub(r"\(;_;"," 😭 ",text)
    text = re.sub(r"\(;_:\)"," 😭 ",text)
    text = re.sub(r"\(;O;\)"," 😭 ",text)
    text = re.sub(r"\(:_;\)"," 😭 ",text)
    text = re.sub(r"\(ToT\)","  😭  ",text)
    text = re.sub(r"Q\.Q"," 😭 ",text)
    text = re.sub(r"T\.T"," 😭 ",text)
    text = re.sub(r"\(-\.-\)"," 😞 ",text)
    text = re.sub(r"\(-_-\)"," 😞 ",text)

    text = re.sub(r"\(一一\)"," 😞 ",text)
    text = re.sub(r"\(；一_一\)"," 😞 ",text)

    text = re.sub(r"\(=\_=\)"," 😩 ",text)
    text = re.sub(r"\(=^\·^=\)"," 😺 ",text)

    text = re.sub(r"\(=^··^=\)"," 😺 ",text)
    text = re.sub(r"=_^= "," 😺 ",text)

    text = re.sub(r"\(\.\.\)"," 😔 ",text)

    text = re.sub(r"\(\._\.\)"," 😔  ",text)
    text = re.sub(r"\(・・\?"," 😕 ",text)
    text = re.sub(r"\(\?_\?\)"," 😕 ",text)

    text = re.sub(r">^_^<"," 😃 ",text)
    text = re.sub(r"<^\!^>"," 😃 ",text)
    text = re.sub(r"^/^","  😃  ",text)
    text = re.sub(r"\(\*^_^\*\)","  😃  ",text)
    text = re.sub(r"\(^^\)","  😃  ",text)
    text = re.sub(r"\(^\.^\)","  😃  ",text)
    text = re.sub(r"\(^_^\.\)","  😃  ",text)
    text = re.sub(r"\(^_^\)","  😃  ",text)
    text = re.sub(r"\(^J^\)","  😃  ",text)
    text = re.sub(r"\(\*^\.^\*\)"," 😃  ",text)
    text = re.sub(r"\(^—^\）"," 😃 ",text)

    text = re.sub(r"\(#^.^#\)"," 😃 ",text)
    text = re.sub(r"\(^—^\)"," 👋 ",text)
    text = re.sub(r"\(;_;\)/~~~","  👋  ",text)

    text = re.sub(r"\(^.^\)/~~~"," 👋 ",text)
    text = re.sub(r"\(-_-\)/~~~ \($··\)/~~~"," 👋 ",text)
    text = re.sub(r"\(T_T\)/~~~"," 👋 ",text)

    text = re.sub(r"\(ToT\)/~~~"," 👋 ",text)
    text = re.sub(r"\(\*^0^\*\)"," 😍 ",text)
    text = re.sub(r"\(\*_\*\)"," 😍 ",text)

    text = re.sub(r"\(\*_\*;"," 😍 ",text)
    text = re.sub(r"\(+_+\) \(@_@\)"," 😍 ",text)
    text = re.sub(r"o\.O"," 😮 ",text)
    text = re.sub(r"\(o\.o\)"," 😮 ",text)

    return text


In [99]:
with open("testBaseline.config") as configfile:
    config = json.load(configfile)

trainDataPath = config["train_data_path"]
testDataPath = config["test_data_path"]
solutionPath = config["solution_path"]
gloveDir = config["glove_dir"]

NUM_FOLDS = config["num_folds"]
NUM_CLASSES = config["num_classes"]
MAX_NB_WORDS = config["max_nb_words"]
MAX_SEQUENCE_LENGTH = config["max_sequence_length"]
EMBEDDING_DIM = config["embedding_dim"]
BATCH_SIZE = config["batch_size"]
LSTM_DIM = config["lstm_dim"]
DROPOUT = config["dropout"]
LEARNING_RATE = config["learning_rate"]
NUM_EPOCHS = config["num_epochs"]

label2emotion = {0:"others", 1:"happy", 2: "sad", 3:"angry"}
emotion2label = {"others":0, "happy":1, "sad":2, "angry":3}

In [194]:
def preprocessData(dataFilePath, mode):
    """Load data from a file, process and return indices, conversations and labels in separate lists
    Input:
        dataFilePath : Path to train/test file to be processed
        mode : "train" mode returns labels. "test" mode doesn't return labels.
    Output:
        indices : Unique conversation ID list
        conversations : List of 3 turn conversations, processed and each turn separated by the <eos> tag
        labels : [Only available in "train" mode] List of labels
    """
    indices = []
    conversations = []
    labels = []
    u1 = []
    u2 = []
    u3 = []
    smilys = []
    
    with io.open(dataFilePath, encoding="utf8") as finput:
        finput.readline()
        for line in finput:
            # Convert multiple instances of . ? ! , to single instance
            # okay...sure -> okay . sure
            # okay???sure -> okay ? sure
            # Add whitespace around such punctuation
            # okay!sure -> okay ! sure
            repeatedChars = ['.', '?', '!', ',']
            for c in repeatedChars:
                lineSplit = line.split(c)
                while True:
                    try:
                        lineSplit.remove('')
                    except:
                        break
                cSpace = ' ' + c + ' '
                line = cSpace.join(lineSplit)

            line = line.strip().split('\t')
            if mode == "train":
                # Train data contains id, 3 turns and label
                label = emotion2label[line[4]]
                labels.append(label)

            conv = ' <eos> '.join(line[1:4])

            #Replace non-unicode smilys with unicode
            conv = str2emoji(conv)

            #Separate smilys w unicode
            conv = add_space(conv)
            
            #smilys            
            just_smilys = remove_text(conv)
            smilys.append(just_smilys)
            
            #remove emoji
            #conv = remove_emoji(conv)
            
            #Many of the words not in embeddings are problematic due to apostrophes e.g. didn't
            #conv = fix_apos(conv)

            # Remove any duplicate spaces
            duplicateSpacePattern = re.compile(r'\ +')
            conv = re.sub(duplicateSpacePattern, ' ', conv)

            #Do the same operations for each turn, remake from before
            u1_line = conv.split(' <eos> ')[0]
            u2_line = conv.split(' <eos> ')[1]
            u3_line = conv.split(' <eos> ')[2]

            u1.append(re.sub(duplicateSpacePattern, ' ', u1_line.lower()))
            u2.append(re.sub(duplicateSpacePattern, ' ', u2_line.lower()))
            u3.append(re.sub(duplicateSpacePattern, ' ', u3_line.lower()))

            indices.append(int(line[0]))
            conversations.append(conv.lower())

    if mode == "train":
        return indices, conversations, labels, u1, u2, u3, smilys
    else:
        return indices, conversations, u1, u2, u3
    
def getEmbeddingMatrix(wordIndex):
    """Populate an embedding matrix using a word-index. If the word "happy" has an index 19,
       the 19th row in the embedding matrix should contain the embedding vector for the word "happy".
    Input:
        wordIndex : A dictionary of (word : index) pairs, extracted using a tokeniser
    Output:
        embeddingMatrix : A matrix where every row has 100 dimensional GloVe embedding
    """
    embeddingsIndex = {}
    # Load the embedding vectors from ther GloVe file
    with io.open(os.path.join(gloveDir, 'glove.6B.100d.txt'), encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            embeddingVector = np.asarray(values[1:], dtype='float32')
            embeddingsIndex[word] = embeddingVector
            
    print('Found %s word vectors.' % len(embeddingsIndex))
    
    #For smilys        
    embeddingsIndexSmily = {}
    with io.open(os.path.join(gloveDir, 'emoji2vec.txt'), encoding="utf8") as f:
        for line in f:
            values = line.split()
            smiley = values[0]
            embeddingVector = np.asarray(values[1:], dtype='float32')
            embeddingsIndexSmily[smiley] = embeddingVector

    print("Found %s smilys" % len(embeddingsIndexSmily))
    
    # Minimum word index of any word is 1.
    embeddingMatrix = np.zeros((len(wordIndex) + 1, EMBEDDING_DIM))
    embeddingSmilyMatrix = np.zeros((len(wordIndex) + 1, 300))
    for word, i in wordIndex.items():
        
        embeddingVector = embeddingsIndex.get(word)
        
        if is_emoji(word):
            embeddingSmilyMatrix[i] = embeddingsIndexSmily.get(word)
        
        if embeddingVector is not None:
            # words not found in embedding index will be all-zeros.
            embeddingMatrix[i] = embeddingVector

    return embeddingMatrix, embeddingSmilyMatrix

In [164]:
indices, conversations, labels, u1, u2, u3, smilys = preprocessData(trainDataPath, 'train')

In [166]:
smiley_data = {'smilys': smilys, 'labels': labels}
df = pd.DataFrame(smiley_data)

In [191]:
embeddingsIndexSmily = {}
with io.open(os.path.join(gloveDir, 'emoji2vec.txt'), encoding="utf8") as f:
    for line in f:
        values = line.split()
        smiley = values[0]
        embeddingVector = np.asarray(values[1:], dtype='float32')
        embeddingsIndexSmily[smiley] = [embeddingVector]

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(u1+u2+u3)
wordIndex = tokenizer.word_index

In [195]:
embeddingMatrix, embeddingSmilyMatrix = getEmbeddingMatrix(wordIndex)

Found 400000 word vectors.
Found 1662 smilys


In [209]:
embeddingSmilyMatrix[1:100]

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.63487846, 0.47953236, 0.64063072, ..., 0.47608215, 0.39532331,
        0.45197266],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])